# Entrenamiento de BERT

### Base de datos

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Cargar el archivo .xlsx
ruta_xlsx = "Base AI.xlsx"
hoja = "Hoja1"

# Leer el archivo Excel
df = pd.read_excel(ruta_xlsx, sheet_name=hoja)

# Guardar como .csv
ruta_csv = "train.csv"  # Reemplaza con la ruta de salida deseada
df.to_csv(ruta_csv, index=False)  # index=False elimina la columna del índice en el .csv

print(f"Archivo convertido a {ruta_csv} exitosamente.")


Archivo convertido a train.csv exitosamente.


### Entrenamiento

In [ ]:
# Paso 11: Instalar los paquetes necesarios para el entrenamiento del modelo
!pip install transformers datasets  # Instalar paquetes transformers y datasets
!pip install transformers[torch]  # Instalar transformers con soporte para PyTorch
!pip install accelerate  # Instalar accelerate
!pip install evaluate  # Instalar evaluate

import pandas as pd
from datasets import load_dataset, Dataset  # Importar Dataset de datasets
from transformers import AutoTokenizer  # Importar AutoTokenizer de transformers
from transformers import AutoModelForSequenceClassification  # Importar AutoModelForSequenceClassification de transformers
from transformers import TrainingArguments, Trainer  # Importar TrainingArguments y Trainer de transformers
import numpy as np  # Importar numpy
import evaluate  # Importar evaluate

In [ ]:
# Paso 12: Cargar el conjunto de datos desde el archivo CSV
dataset = pd.read_csv("train.csv")
dataset.head()

,Nombre Noticia,Categoría en finanzas personales,URL,Descripción
0,"El dólar cerró al alza, después del anuncio de...",4,https://www.larepublica.co/finanzas/precios-de...,"Cuando el dólar sube, los productos importados..."
1,¿Cómo saber si tengo devolución de impuestos e...,3,https://www.portafolio.co/economia/finanzas/co...,Saber si tienes una devolución de impuestos en...
2,Salario mínimo 2025: así quedaría con el aumen...,3,https://www.portafolio.co/economia/finanzas/sa...,Es relevante para las finanzas personales porq...
3,Amenaza arancelaria de Trump a los socios come...,2,https://www.larepublica.co/globoeconomia/amena...,El anuncio de aranceles adicionales por parte ...
4,Contraloría General lanza alerta sobre la afec...,2,https://www.larepublica.co/economia/contralori...,La alerta sobre una posible desfinanciación de...


In [ ]:
dataset = dataset.rename(columns={'Categoría en finanzas personales ': 'label', 'Descripción ': 'text'})
dataset = dataset.drop(columns=['URL', "Nombre Noticia "])
dataset.head()

,label,text
0,4,"Cuando el dólar sube, los productos importados..."
1,3,Saber si tienes una devolución de impuestos en...
2,3,Es relevante para las finanzas personales porq...
3,2,El anuncio de aranceles adicionales por parte ...
4,2,La alerta sobre una posible desfinanciación de...


In [ ]:
# Paso 13: Dividir el conjunto de datos en conjuntos de entrenamiento y evaluación
from sklearn.model_selection import train_test_split

# Asumiendo que tienes columnas llamadas "text" y "label"
train_texts, eval_texts, train_labels, eval_labels = train_test_split(dataset["text"], dataset["label"], test_size=0.2, random_state=42)

In [ ]:
print(train_texts, eval_texts, train_labels, eval_labels)

28     La disminución en la utilidad neta y el EBITD...
24    La restructuración de deuda y las mejores pers...
12    Si formas parte de la industria del entretenim...
0     Cuando el dólar sube, los productos importados...
4     La alerta sobre una posible desfinanciación de...
16    Si estás interesado en invertir, estar al tant...
5     Es importante para las finanzas personales, es...
13    Las personas con inversiones en criptomonedas ...
11    Las calificaciones AAA y VrR 1+ indican que Sc...
22    Aquellos que están interesados en optimizar su...
1     Saber si tienes una devolución de impuestos en...
2     Es relevante para las finanzas personales porq...
25    i solo dos ramas de la economía están impulsan...
3     El anuncio de aranceles adicionales por parte ...
21    Los productores locales, ganaderos y trabajado...
26    Las personas pueden considerar incluir accione...
18    Para los accionistas de Cementos Argos, este p...
29    La inflación en Bogotá durante 2023 fue de

In [ ]:
# Crear diccionarios para los conjuntos de entrenamiento y evaluación
train_dataset = {"text": train_texts, "label": train_labels}
eval_dataset = {"text": eval_texts, "label": eval_labels}

# Convertir diccionarios a objetos Dataset
train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

print(train_dataset, eval_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 24
}) Dataset({
    features: ['text', 'label'],
    num_rows: 6
})


In [ ]:
# Paso 14: Cargar el tokenizador BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # Cargar tokenizador BERT

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Paso 15: Definir la función para tokenizar
def tokenize_function(examples):  # Definir función para tokenizar
    """

    Args:
      examples:

    Returns:

    """
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [ ]:
# Paso 16: Aplicar tokenización al conjunto de datos
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de entrenamiento
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de evaluación

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
# Paso 18: Cargar el modelo BERT para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=len(dataset["label"].unique()))  # Cargar modelo BERT para clasificación de secuencias


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Paso 19: Definir hiperparámetros de entrenamiento
training_args = TrainingArguments(output_dir="restaurants-reviews", evaluation_strategy="epoch", report_to="none")  # Definir hiperparámetros de entrenamiento con evaluación por época


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Paso 20: Cargar la métrica de precisión
metric = evaluate.load("accuracy")  # Cargar métrica de precisión

In [ ]:
# Paso 21: Definir la función para calcular métricas
def compute_metrics(eval_pred):  # Definir función para calcular métricas
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Paso 22: Crear el objeto Trainer (continuación)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)  # Crear objeto Trainer

In [ ]:
# Paso 23: Entrenar el modelo
trainer.train()  # Entrenar el modelo

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.558811,0.333333
2,No log,1.583115,0.333333
3,No log,1.615273,0.333333


TrainOutput(global_step=9, training_loss=1.5909784105088975, metrics={'train_runtime': 513.5874, 'train_samples_per_second': 0.14, 'train_steps_per_second': 0.018, 'total_flos': 18944506257408.0, 'train_loss': 1.5909784105088975, 'epoch': 3.0})

In [ ]:
# Paso 24: Importar el pipeline de transformers
from transformers import pipeline  # Importar pipeline de transformers

# Paso 25: Definir un prompt de ejemplo
prompt = "holaaaaaaaaahgilbv"

# Paso 26: Crear el pipeline de clasificación de texto
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)  # Crear pipeline de clasificación de texto

# Paso 27: Realizar inferencia con el texto de ejemplo
result = pipe(f"[INST] {prompt} [/INST] \n\n")  # Realizar inferencia con texto de ejemplo

# Paso 28: Imprimir el resultado
print(result)  # Imprimir resultado


[{'label': 'LABEL_2', 'score': 0.320701003074646}]


In [ ]:
# Paso 29: Mejorar la respuesta del modelo

# Diccionario para convertir etiquetas numéricas a etiquetas originales
etiquetas_originales = {"LABEL_0": "Muy Malo", "LABEL_1": "Malo", "LABEL_2": "Neutro", "LABEL_3": "Bueno", "LABEL_4": "Muy Bueno"}

# Paso 30: Definir la función para convertir predicciones
def convertir_predicciones(predicciones):
    etiquetas_predichas = []
    for prediccion in predicciones:
        etiqueta_predicha = None
        for clave, valor in etiquetas_originales.items():
            if clave == prediccion:
                etiqueta_predicha = valor
                break
        if etiqueta_predicha is None:
            etiqueta_predicha = prediccion
        etiquetas_predichas.append(etiqueta_predicha)
    return etiquetas_predichas

In [ ]:
# Paso 31: Ejemplo de inferencia y conversión de predicciones
prompt = "Informe del IRS 2024 reveló riesgos y oportunidades para la inversión en las regiones"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)
result = pipe(prompt)[0]
prediccion_numerica = result['label']
etiqueta_predicha = convertir_predicciones([prediccion_numerica])[0]
print(f"Predicción: {etiqueta_predicha}")

Predicción: Neutro


# Aplicación en interfaz

In [ ]:
!pip install gradio transformers torch scikit-learn

Parcial Final

In [ ]:
!pip install streamlit beautifulsoup4 spacy plotly requests
!python -m spacy download es_core_news_sm
!pip install gradio requests spacy plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 45.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Importar las librerías necesarias
from transformers import pipeline
import gradio as gr
import plotly.graph_objects as go

# Función para calcular impacto financiero
def calcular_impacto_financiero(ingreso, ahorros, deudas, indicators):
    impacto = {
        'ahorros': 0,
        'deudas': 0,
        'ingreso_disponible': ingreso
    }
    if indicators.get('tasa_interes'):
        try:
            tasa = float(indicators['tasa_interes'][0].replace('%', '')) / 100
            impacto['ahorros'] = ahorros * tasa
            impacto['deudas'] = deudas * (tasa * 1.5)
            impacto['ingreso_disponible'] = ingreso - impacto['deudas']
        except:
            pass
    return impacto

# Función para generar informe
def generar_informe(ingreso, ahorros, deudas, impacto, indicators):
    informe = f"""
## Análisis de Impacto Económico

### Indicadores Económicos
•⁠  ⁠Tasas de interés: {indicators.get('tasa_interes', 'No detectadas')}
•⁠  ⁠Inflación: {indicators.get('inflacion', 'No detectada')}
•⁠  ⁠Divisas: {indicators.get('cambio_divisas', 'No detectadas')}

### Impacto Financiero
•⁠  ⁠Ingreso Original: {ingreso:.2f}
•⁠  ⁠Ingreso Disponible Estimado: {impacto['ingreso_disponible']:.2f}
•⁠  ⁠Cambio en Ahorros: {impacto['ahorros']:.2f}
•⁠  ⁠Cambio en Deudas: {impacto['deudas']:.2f}

### Recomendaciones
1.⁠ ⁠Monitorear cambios económicos.
2.⁠ ⁠Revisar estrategia financiera.
3.⁠ ⁠Considerar ajustes en presupuesto.
"""
    return informe

# Función principal que integra análisis textual y financiero
def analizar_impacto_integrado(prompt, ingreso, ahorros, deudas):
    # Modelo de clasificación de texto
    pipe = pipeline(task="text-classification", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")
    result = pipe(prompt)[0]

    # Predicción numérica a etiqueta
    prediccion_numerica = result['label']
    etiquetas = {
        'LABEL_0': 'Muy Negativa',
        'LABEL_1': 'Negativa',
        'LABEL_2': 'Neutral',
        'LABEL_3': 'Positiva',
        'LABEL_4': 'Muy Positiva'
    }
    etiqueta_predicha = etiquetas.get(prediccion_numerica, 'Etiqueta Desconocida')

    # Simular extracción de indicadores económicos
    indicators = {
        'tasa_interes': ['5%'],
        'inflacion': ['3%'],
        'cambio_divisas': ['USD/COP: 4000']
    }

    # Calcular impacto financiero
    impacto = calcular_impacto_financiero(ingreso, ahorros, deudas, indicators)

    # Generar informe
    informe = generar_informe(ingreso, ahorros, deudas, impacto, indicators)

    # Crear visualización financiera
    fig = go.Figure(data=[
        go.Bar(name='Antes', x=['Ingreso', 'Ahorros', 'Deudas'], y=[ingreso, ahorros, deudas]),
        go.Bar(name='Después', x=['Ingreso', 'Ahorros', 'Deudas'],
               y=[impacto['ingreso_disponible'], ahorros + impacto['ahorros'], deudas + impacto['deudas']])
    ])
    fig.update_layout(barmode='group', title="Comparación Financiera")

    # Resultado combinado
    resultado = f"*Predicción Textual:* {etiqueta_predicha}\n\n{informe}"
    return resultado, fig

# Crear la interfaz integrada de Gradio
demo = gr.Interface(
    fn=analizar_impacto_integrado,
    inputs=[
        gr.Textbox(label="Introduce el texto del informe económico"),
        gr.Number(label="Ingreso mensual"),
        gr.Number(label="Ahorros"),
        gr.Number(label="Deudas")
    ],
    outputs=[
        gr.Textbox(label="Resultado del Análisis"),
        gr.Plot(label="Visualización del Impacto Financiero")
    ],
    title="Análisis Económico Integrado",
    description="Esta herramienta combina análisis de impacto económico textual con cálculo de impacto financiero para decisiones informadas."
)

# Lanzar la interfaz
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4029d9a4f419f2822.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
